In [196]:
import pandas as pd
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import seaborn as sns
from scipy.stats import spearmanr

init_notebook_mode(connected=True)

In [197]:
df = pd.read_csv('final_train.csv', index_col=0)

In [198]:
df

,Activity,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),"fBodyAcc-bandsEnergy()-1,16",...,tGravityAccMag-energy(),tGravityAccMag-entropy(),tGravityAccMag-iqr(),tGravityAccMag-mad(),tGravityAccMag-max(),tGravityAccMag-mean(),tGravityAccMag-min(),tGravityAccMag-sma(),tGravityAccMag-std(),void()
0,STANDING,-1.574191,-0.805700,-2.453831,-1.861714,0.115517,-2.023369,3.125860,-9.474436,3.622600,...,5.049130,3.272281,5.103018,-0.823302,3.063681,-0.928676,-1.007695,3.094236,1.669987,1.0
1,WALKING_UPSTAIRS,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,2.512880,...,4.565443,4.440779,3.288399,-2.689605,1.705341,-0.822617,-0.572410,-2.187890,-0.544652,2.0
2,LAYING,-1.548333,-0.139628,-2.380889,-1.915239,0.067613,-0.887342,6.007361,-11.906904,3.621984,...,5.059609,2.296603,5.344174,-0.549843,3.272225,-0.940610,-1.031522,-2.071755,2.024442,3.0
3,WALKING,-1.574094,-0.793443,-2.449959,-1.842741,-0.195625,-0.950671,2.104353,-11.906121,3.355701,...,4.860758,4.177535,3.950012,-0.322642,2.366287,-0.866937,-0.696431,1.863375,0.550414,2.0
4,LAYING,-1.548333,-0.139628,-2.380889,NaN,0.067613,-0.887342,6.007361,-11.970485,3.621984,...,5.059609,2.296603,5.344174,-0.549843,3.272225,-0.940610,-1.031522,-2.102513,2.024442,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11081,WALKING_UPSTAIRS,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,2.512880,...,4.565443,4.440779,3.288399,-2.689605,1.705341,-0.822617,-0.572410,3.534816,-0.544652,2.0
11082,LAYING,-1.550288,-0.231565,-2.376561,-1.868524,0.007653,-0.077020,5.494466,-5.860692,3.603169,...,5.048737,2.923856,5.182026,-0.998495,2.853755,-0.933561,-1.000173,-1.984575,1.372763,1.0
11083,WALKING_UPSTAIRS,-1.569815,-0.794940,-2.482325,-1.784151,0.021844,-0.740329,8.005261,-11.916107,2.644861,...,4.453514,4.302852,2.979961,-6.032382,1.853029,-0.802817,0.041975,3.427548,-0.075184,1.0
11084,WALKING_DOWNSTAIRS,-1.574768,-0.752847,-2.449529,-1.737755,-0.189761,-0.454246,8.054443,-5.611277,2.473134,...,4.615939,4.443277,3.208597,-6.200222,1.675269,-0.828934,-0.877004,3.385405,-0.383217,3.0


In [199]:
sns.set()
plt.figure(figsize=(16, 12))
px.histogram(df['Activity'])

<Figure size 1152x864 with 0 Axes>

In [200]:
from sklearn.metrics import mutual_info_score

df = df.fillna(df.mean())

def rank_features(df, features, labels, fun, index=None):
    '''
    fun - function used to calculate the informativity of features
    index - optional, if the function returns a result in a tuple/list, then index specifies at which 
            index the considreredmetric is found 
    '''
    #df_results = pd.DataFrame(columns = ['Feature, Informativity'])
    results = []
    for feature in features:
        result = fun(df[feature], labels)
        if index is not None:
            result = abs(result[index])
        results.append({'Feature': feature, 'Informativity': result})
        
    return pd.DataFrame(results).sort_values(by='Informativity').reset_index(drop=True)


features = df.columns[2:]
labels = df['Activity']


spearman = rank_features(df, features, labels, spearmanr, index=0)
spearman   

,Feature,Informativity
0,fBodyAcc-maxInds-Y,0.000889
1,tGravityAcc-entropy()-X,0.009404
2,"tGravityAcc-arCoeff()-X,3",0.016574
3,tBodyAccJerk-mean()-Z,0.017079
4,"tBodyGyroJerk-arCoeff()-X,3",0.022492
...,...,...
565,fBodyAcc-mad()-Z,0.813794
566,"fBodyAcc-bandsEnergy()-1,24.2",0.820358
567,fBodyAcc-std()-Z,0.828246
568,"fBodyAcc-bandsEnergy()-1,16.2",0.833516


In [201]:
mutual_info = rank_features(df, features, labels, mutual_info_score)
mutual_info

,Feature,Informativity
0,void(),0.106098
1,subject,0.342704
2,fBodyGyro-maxInds-X,0.409109
3,fBodyAccJerk-maxInds-Z,0.422734
4,fBodyAccJerk-maxInds-Y,0.442125
...,...,...
565,"tBodyGyroJerk-arCoeff()-X,2",1.774695
566,fBodyAccJerk-skewness()-X,1.774827
567,fBodyAcc-kurtosis()-Z,1.775045
568,fBodyAcc-main()-Z,1.775231


In [202]:
spearman.loc[spearman['Feature'] == 'void()']

,Feature,Informativity
81,void(),0.223558


In [203]:
activities = list(df['Activity'].unique())
activities_codes = {label:i for i, label in enumerate(activities)}
activities_codes

{'STANDING': 0,
 'WALKING_UPSTAIRS': 1,
 'LAYING': 2,
 'WALKING': 3,
 'SITTING': 4,
 'WALKING_DOWNSTAIRS': 5}

In [204]:
mutual_info.head(50)

,Feature,Informativity
0,void(),0.106098
1,subject,0.342704
2,fBodyGyro-maxInds-X,0.409109
3,fBodyAccJerk-maxInds-Z,0.422734
4,fBodyAccJerk-maxInds-Y,0.442125
5,fBodyBodyGyroMag-maxInds,0.470028
6,fBodyGyro-maxInds-Y,0.486909
7,fBodyBodyGyroJerkMag-maxInds,0.511472
8,fBodyBodyAccJerkMag-maxInds,0.520590
9,fBodyGyro-maxInds-Z,0.567642


In [205]:
px.line(spearman, x='Feature', y='Informativity')

In [206]:
px.histogram(spearman, x='Feature', y='Informativity')

In [207]:
px.line(mutual_info, x='Feature', y='Informativity')

In [208]:
from sklearn import preprocessing



X = df.drop(['Activity', 'void()', 'subject'], axis='columns')
X = X.fillna(X.mean)


def normalize(df):
    cols = df.columns
    x = df.values #returns a numpy array
    min_max_scaler = preprocessing.StandardScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return pd.DataFrame(x_scaled, columns=cols)

# X = normalize(X)

In [209]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#model_lr = LogisticRegression(solver='liblinear', multi_class='auto').fit(X_train, y_train)
model_lr = LogisticRegression(solver='liblinear', multi_class='ovr').fit(X_train, y_train)

In [210]:
model_lr.score(X_test, y_test)

0.9759470835838846

In [211]:
from statistics import mean
from sklearn.model_selection import KFold

def sk_k_fold_validation(X, y, k, model_constructor, *model_args, **model_kwargs):
    k_fold = KFold(n_splits=k)
    scores = []
    for train_idx, test_idx in k_fold.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model = model_constructor(*model_args, **model_kwargs).fit(X=X_train, y=y_train)
        scores.append(model.score(X_test, y_test))
        
    return mean(scores)
        

sk_k_fold_validation(X.values, y, k=5, model_constructor=LogisticRegression, solver='liblinear', multi_class='ovr')

0.9807866746547805

In [212]:
from sklearn.svm import SVC

model_svc = SVC().fit(X_train, y_train)
model_svc.score(X_test, y_test)

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:193: FutureWarning:

The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.



0.9639206253758268

In [213]:
from sklearn.neural_network import MLPClassifier

model_mlp = MLPClassifier().fit(X_train, y_train)
model_mlp.score(X_test, y_test)

0.9693325315694528

In [214]:
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier().fit(X_train, y_train)
model_tree.score(X_test, y_test)

0.9555021046301864

In [215]:
from sklearn import tree
import numpy as np

#np.std(X_train, 0)*model_lr.coef_

importance = abs(model_lr.coef_[0])
# summarize feature importance
# for i,v in enumerate(importance):
#     print('Feature: %0d, Score: %.5f' % (i,v))
    

importance_df = pd.DataFrame(importance, columns=['Importance'])
importance_df.sort_values(by='Importance')


,Importance
263,0.000016
317,0.000041
14,0.000084
349,0.000150
556,0.000259
...,...
559,0.580719
315,0.711596
532,1.046112
543,2.427757


In [216]:
X.columns[283]
#model_lr.classes_

'fBodyGyro-maxInds-Z'

In [217]:
px.bar([x for x in range(len(importance))], importance)

In [218]:
from plotly.subplots import make_subplots


def most_important_features(X, model_lr, top_n=5):
    
    feature_importance = {}

    for i, aclass in enumerate(model_lr.classes_):
        importance = abs(model_lr.coef_[i])

        importance_df = pd.DataFrame(importance, columns=['Importance'])
        importance_df = importance_df.sort_values(by='Importance',ascending=False)
        top = importance_df[:top_n]
        top.index.name = 'Feature'
        top = top.reset_index()
        top['Feature'] = top['Feature'].map(lambda i: X.columns[i])
        #best_features = {X.columns[col]:importance for col, importance in top.itertuples()}
        feature_importance[aclass] = top
        
    return feature_importance


def plot_most_important_features(feature_importance):
    plt_titles = list(feature_importance.keys())
    length = len(feature_importance)
    
    fig = make_subplots(
        rows=len(feature_importance),
        cols=1,
        subplot_titles=plt_titles,
    )
    
    for i,top in enumerate(feature_importance.values()):
        f = px.bar(top, x='Feature', y='Importance')
        fig.add_trace(f.data[0], row = i + 1, col = 1)
    fig.update_layout(height=length * 350, width=900)
    #fig.update(layout_coloraxis_showscale=False)
    fig.show()

feature_importance = most_important_features(X, model_lr, top_n=8)

In [219]:
plot_most_important_features(feature_importance)